In [ ]:
!git clone https://github.com/prateeksaxena2809/EPIE_Corpus.git

Cloning into 'EPIE_Corpus'...
remote: Enumerating objects: 42, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 42 (delta 12), reused 21 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (42/42), 1.73 MiB | 4.63 MiB/s, done.
Resolving deltas: 100% (12/12), done.


In [ ]:
try:
    import google.colab # type: ignore
    IN_COLAB = True
except:
    IN_COLAB = False

import os, sys
chapter = "chapter1_transformer_interp"
repo = "ARENA_3.0"

if IN_COLAB:
    # Install packages
    %pip install transformer_lens
    %pip install einops
    %pip install jaxtyping
    %pip install git+https://github.com/callummcdougall/CircuitsVis.git#subdirectory=python

    # Code to download the necessary files (e.g. solutions, test funcs)
    if not os.path.exists(f"/content/{chapter}"):
        !wget https://github.com/callummcdougall/ARENA_3.0/archive/refs/heads/arena_pre_v4.zip
        !unzip /content/arena_pre_v4.zip 'ARENA_3.0-arena_pre_v4/chapter1_transformer_interp/exercises/*'
        sys.path.append(f"/content/{repo}-arena_pre_v4/{chapter}/exercises")
        os.remove("/content/arena_pre_v4.zip")
        os.rename(f"{repo}-arena_pre_v4/{chapter}", chapter)
        os.rmdir(f"{repo}-arena_pre_v4")
        os.chdir(f"{chapter}/exercises")
else:
    raise Exception("If running from VSCode, you should copy code from the Streamlit page, not the Colab.")

  Cloning https://github.com/callummcdougall/CircuitsVis.git to /tmp/pip-req-build-br5h7bbk
  Running command git clone --filter=blob:none --quiet https://github.com/callummcdougall/CircuitsVis.git /tmp/pip-req-build-br5h7bbk
  Resolved https://github.com/callummcdougall/CircuitsVis.git to commit 1e6129d08cae7af9242d9ab5d3ed322dd44b4dd3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import sys, os
import re
import torch as t
from torch import Tensor
import torch.nn as nn
import torch.nn.functional as F
from pathlib import Path
import numpy as np
import einops
from jaxtyping import Int, Float
import functools
from tqdm import tqdm
from IPython.display import display
from transformer_lens.hook_points import HookPoint
from transformer_lens import (
    utils,
    HookedTransformer,
    HookedTransformerConfig,
    FactoredMatrix,
    ActivationCache,
)
import circuitsvis as cv

# Make sure exercises are in the path
chapter = "chapter1_transformer_interp"
#repo = "ARENA_3.0"
exercises_dir = Path(f"{os.getcwd().split(chapter)[0]}/{chapter}/exercises").resolve()
section_dir = (exercises_dir / "part2_intro_to_mech_interp").resolve()
if str(exercises_dir) not in sys.path: sys.path.append(str(exercises_dir))

from plotly_utils import imshow, hist, plot_comp_scores, plot_logit_attribution, plot_loss_difference
from part1_transformer_from_scratch.solutions import get_log_probs
import part2_intro_to_mech_interp.tests as tests

# Saves computation time, since we don't need it for the contents of this notebook
t.set_grad_enabled(False)

device = t.device("cuda" if t.cuda.is_available() else "cpu")

MAIN = __name__ == "__main__"

In [ ]:
pythia: HookedTransformer = HookedTransformer.from_pretrained("EleutherAI/pythia-14m")

In [ ]:
pythia_14m: HookedTransformer = HookedTransformer.from_pretrained("EleutherAI/pythia-14m")
pythia_70m: HookedTransformer = HookedTransformer.from_pretrained("EleutherAI/pythia-70m")
pythia_1_4b: HookedTransformer = HookedTransformer.from_pretrained("EleutherAI/pythia-1.4b")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The `GPTNeoXSdpaAttention` class is deprecated in favor of simply modifying the `config._attn_implementation`attribute of the `GPTNeoXAttention` class! It will be removed in v4.48


Loaded pretrained model EleutherAI/pythia-14m into HookedTransformer
Loaded pretrained model EleutherAI/pythia-70m into HookedTransformer
Loaded pretrained model EleutherAI/pythia-1.4b into HookedTransformer


In [ ]:
# Optional
pythia_14m.cfg

HookedTransformerConfig:
{'NTK_by_parts_factor': 8.0,
 'NTK_by_parts_high_freq_factor': 4.0,
 'NTK_by_parts_low_freq_factor': 1.0,
 'act_fn': 'gelu',
 'attention_dir': 'causal',
 'attn_only': False,
 'attn_scale': 5.656854249492381,
 'attn_scores_soft_cap': -1.0,
 'attn_types': None,
 'checkpoint_index': None,
 'checkpoint_label_type': None,
 'checkpoint_value': None,
 'd_head': 32,
 'd_mlp': 512,
 'd_model': 128,
 'd_vocab': 50304,
 'd_vocab_out': 50304,
 'decoder_start_token_id': None,
 'default_prepend_bos': True,
 'device': device(type='cuda'),
 'dtype': torch.float32,
 'eps': 1e-05,
 'experts_per_token': None,
 'final_rms': False,
 'from_checkpoint': False,
 'gated_mlp': False,
 'init_mode': 'gpt2',
 'init_weights': False,
 'initializer_range': 0.07071067811865475,
 'load_in_4bit': False,
 'model_name': 'pythia-14m',
 'n_ctx': 2048,
 'n_devices': 1,
 'n_heads': 4,
 'n_key_value_heads': None,
 'n_layers': 6,
 'n_params': 1179648,
 'normalization_type': 'LNPre',
 'num_experts': None

In [ ]:
# DATA
with open("/content/EPIE_Corpus/Formal_Idioms_Corpus/Formal_Idioms_Labels.txt", 'r', encoding = "utf-8") as lbl_file:
  lbls = lbl_file.readlines()

# print(len(lbls))
# print(lbls)

with open("/content/EPIE_Corpus/Formal_Idioms_Corpus/translated_sentences.txt", 'r', encoding = "utf-8") as trans_file:
  trans = trans_file.readlines()

# print(len(trans))
# print(trans)

with open("/content/EPIE_Corpus/Formal_Idioms_Corpus/Formal_Idioms_Words.txt", 'r', encoding = "utf-8") as idiom_file:
  idioms = idiom_file.readlines()

# print(len(idioms))
# print(idioms)

tokenized_idiom_sents = [idioms[i].strip() for i in range(len(lbls)) if lbls[i].strip() == '1']
tokenized_literal_sents = [idioms[i].strip() for i in range(len(lbls)) if lbls[i].strip() == '0']
tokenized_trans_idiom_sents = [trans[i].strip() for i in range(len(lbls)) if lbls[i].strip() == '1']
tokenized_trans_literal_sents = [trans[i].strip() for i in range(len(lbls)) if lbls[i].strip() == '0']

print(f"{len(tokenized_idiom_sents)} formal idioms:")
print(tokenized_idiom_sents)

# print(f"{len(literal_sents)} literal idioms:")
# print(literal_sents)

print(f"{len(tokenized_trans_idiom_sents)} translated idioms:")
print(tokenized_trans_idiom_sents)

# print(len(trans_literal_sents))
# print(trans_literal_sents)

2761 formal idioms:
['‘ You know , the panda who keeps an eye on my drinking habits . ’', 'But I will also be keeping an eye on you . ’', '‘ I will keep an eye on him , ’ reassured Jack .', "Moreover , international operations director Richard Ferry claims that the Bracknell , Berkshire-based company 's services are unique — although most rivals can offer centralised expertise , he says , no-one else can monitor computer systems remotely , or keep an eye on such critical issues as temperature and air conditioning levels at any given customer site .", 'Once Duncan had managed to fire off a report and stop his watch-replacement from stumbling on the terrorists , he had settled down to keep an eye on the gang .', 'So we must keep an eye on Bruno Casanova . ’', 'Keep it in the quarantine tank , cease medication and keep an eye on it for the next few months .', '‘ As you can see from the state of this place , I have someone call in regularly to keep an eye on it .', 'Mary organized the plat

In [ ]:
# Remove spaces
def remove_spaces(sent_list):
  cleaned_sents = []
  for sent in sent_list:
    space_matches = set(re.findall(r"(‘ | ['’\.,]| $)", sent))
    del_space_matches = {space_match:space_match.replace(' ', '') for space_match in space_matches}

    for space_match, del_space in del_space_matches.items():
      sent = sent.replace(space_match, del_space)
    cleaned_sents.append(sent)
  return cleaned_sents

idiom_sents = remove_spaces(tokenized_idiom_sents)
trans_sents = remove_spaces(tokenized_trans_idiom_sents)

print(idiom_sents)
print(trans_sents)

['‘You know, the panda who keeps an eye on my drinking habits.’', 'But I will also be keeping an eye on you.’', '‘I will keep an eye on him,’ reassured Jack.', "Moreover, international operations director Richard Ferry claims that the Bracknell, Berkshire-based company's services are unique — although most rivals can offer centralised expertise, he says, no-one else can monitor computer systems remotely, or keep an eye on such critical issues as temperature and air conditioning levels at any given customer site.", 'Once Duncan had managed to fire off a report and stop his watch-replacement from stumbling on the terrorists, he had settled down to keep an eye on the gang.', 'So we must keep an eye on Bruno Casanova.’', 'Keep it in the quarantine tank, cease medication and keep an eye on it for the next few months.', '‘As you can see from the state of this place, I have someone call in regularly to keep an eye on it.', 'Mary organized the plates and kept an eye on his carving.', 'Keep an 

In [ ]:
# LOSS
# Optional
# single example
spaces_sent = "‘ Give you thirty seconds , Archie , to spill the beans , ’ said Joe , who was still guessing . "
idiom_sent = "‘Give you thirty seconds, Archie, to spill the beans,’ said Joe, who was still guessing."
literal_sent = "I always spill the drinks ."
translated_sent = "‘ Give you thirty seconds , Archie , to reveal the secret , ’ said Joe , who was still guessing ."

spaces_loss = pythia(spaces_sent, return_type="loss")
idiom_loss = pythia(idiom_sent, return_type="loss")
literal_loss = pythia(literal_sent, return_type="loss")
translated_loss = pythia(translated_sent, return_type="loss")

print(f"Loss of the idiom sentence with spaces '{spaces_sent}': {spaces_loss}")
print(f"Loss of the idiom sentence without spaces '{idiom_sent}': {idiom_loss}")
print(f"Loss of the literal sentence '{literal_sent}': {literal_loss}")
print(f"Loss of the translated sentence '{translated_sent}': {translated_loss}")

Loss of the idiom sentence with spaces '‘ Give you thirty seconds , Archie , to spill the beans , ’ said Joe , who was still guessing . ': 8.636012077331543
Loss of the idiom sentence without spaces '‘Give you thirty seconds, Archie, to spill the beans,’ said Joe, who was still guessing.': 6.16881799697876
Loss of the literal sentence 'I always spill the drinks .': 8.988242149353027
Loss of the translated sentence '‘ Give you thirty seconds , Archie , to reveal the secret , ’ said Joe , who was still guessing .': 8.359539985656738


In [ ]:
# all texts
def calculate_losses(model):
  print("Model:", model.cfg.model_name)
  idiom_losses = []
  trans_losses = []
  for i in range(len(idiom_sents)):
    idiom_losses.append(model(idiom_sents[i], return_type="loss"))
    trans_losses.append(model(trans_sents[i], return_type="loss"))
  #print("Idiom losses: ", idiom_losses)
  #print("Trans losses: ", trans_losses)

  num_sents = len(idiom_sents)
  avg_idiom_loss = sum(idiom_losses)/num_sents
  avg_trans_loss = sum(trans_losses)/num_sents
  max_idiom_loss = max(idiom_losses)
  max_trans_loss = max(trans_losses)
  min_trans_loss = min(trans_losses)
  inst_differences = [idiom_losses[i] - trans_losses[i] for i in range(len(idiom_losses))]
  max_inst_differences = max(inst_differences)
  max_inst_diff_pos = inst_differences.index(max_inst_differences)

  print(f"The average idiom loss is: {avg_idiom_loss}.")
  print(f"The average translated sentence loss is: {avg_trans_loss}.")
  print(f"The idiom with the highest loss is: {idiom_sents[idiom_losses.index(max_idiom_loss)]}.")
  print(f"The translated sentence with the highest loss is: {trans_sents[trans_losses.index(max_trans_loss)]}.")
  print(f"The average difference between instances is: {sum(inst_differences)/num_sents}.")
  print(f"The biggest difference is {max_idiom_loss-min_trans_loss} lies between the sentences\nIdiom: {max_idiom_loss} - {idiom_sents[idiom_losses.index(max_idiom_loss)]}\nand\nTranslated Sentence: {min_trans_loss} - {trans_sents[trans_losses.index(min_trans_loss)]}")
  print(f"The biggest difference between instances is {max_inst_differences} for the sentences\nIdiom: {idiom_losses[max_inst_diff_pos]} - {idiom_sents[max_inst_diff_pos]}\nand\nTranslated Sentence: {trans_losses[max_inst_diff_pos]} - {trans_sents[max_inst_diff_pos]}")

In [ ]:
# Optional
calculate_losses(pythia_14m)

Model: pythia-14m
The average idiom loss is: 6.307247161865234.
The average translated sentence loss is: 6.420248031616211.
The idiom with the highest loss is: saving face.
The translated sentence with the highest loss is: lying.
The average difference between instances is: -0.11301830410957336.
The biggest difference is 14.14580249786377 lies between the sentences
Idiom: 17.954404830932617 - saving face
and
Translated Sentence: 3.8086025714874268 - I hope they will be able to understand my view just as I will be open to their views and opinions about this issue.
The biggest difference between instances is 8.374486923217773 for the sentences
Idiom: 14.247894287109375 - Bag ladies.
and
Translated Sentence: 5.873407363891602 - homeless women who carries all of her personal belongings with her in bags.


In [ ]:
# Optional
calculate_losses(pythia_70m)

Model: pythia-70m
The average idiom loss is: 4.992804527282715.
The average translated sentence loss is: 5.205415725708008.
The idiom with the highest loss is: Party animal.
The translated sentence with the highest loss is: lying.
The average difference between instances is: -0.2126113474369049.
The biggest difference is 8.456331253051758 lies between the sentences
Idiom: 11.484151840209961 - Party animal
and
Translated Sentence: 3.027820587158203 - We believe that it is not fair, it is not practical, and it will not endure for a very long time.
The biggest difference between instances is 4.817378520965576 for the sentences
Idiom: 9.939691543579102 - Japan trips the light fantastic
and
Translated Sentence: 5.122313022613525 - Japan TRIPS THE LIGHT FANTASTIC


In [ ]:
# Optional
calculate_losses(pythia_1_4b)

Model: pythia-1.4b
The average idiom loss is: 4.099091529846191.
The average translated sentence loss is: 4.482586860656738.
The idiom with the highest loss is: Home truth.
The translated sentence with the highest loss is: lying.
The average difference between instances is: -0.38349878787994385.
The biggest difference is 6.847797393798828 lies between the sentences
Idiom: 9.326918601989746 - Home truth
and
Translated Sentence: 2.479121208190918 - The Centre for Alternative Technology ( CAT ) has worked out that, on average, 20 per cent of heating GOES THROUGH THE ROOF, 25 per cent through the walls, 10 per cent through the floor, 10 per cent through the windows and 10 per cent in draughts.
The biggest difference between instances is 4.267062187194824 for the sentences
Idiom: 8.508241653442383 - Bag ladies.
and
Translated Sentence: 4.241179466247559 - homeless women who carries all of her personal belongings with her in bags.


In [ ]:
# PROMPTING
# Data preparation
tokens = [sent.split(" ") for sent in tokenized_idiom_sents]

with open("/content/EPIE_Corpus/Formal_Idioms_Corpus/Formal_Idioms_Tags.txt", 'r', encoding = "utf-8") as tag_file:
  tags = tag_file.readlines()

tags = [tags[i].strip().split(" ") for i in range(len(lbls)) if lbls[i].strip() == '1']
print(tokens)
print(tags)
print(len(tags))

idiom_tag = "I-IDIOM"
inputs = []
correct_toks = []

for (pos, sent) in tqdm(enumerate(tags), desc= "Generating inputs and correct tokens"):
  #print("pos:", pos)
  index = None
  toks = tokens[pos]

  if "I-IDIOM" not in sent:
    index = sent.index("B-IDIOM")
    correct_toks.append(toks[index])
    inputs.append(toks[:index])
  else:
    reverse_tags = sent[::-1]
    reverse_toks = toks[::-1]
    index = reverse_tags.index("I-IDIOM")

    # print("reverse_tags", reverse_tags)
    # print("reverse_toks", reverse_toks)
    # print("index", index)

    correct_toks.append(reverse_toks[index])
    input = reverse_toks[index+1:]

    inputs.append(remove_spaces([" ".join(input[::-1])])[0])

print(correct_toks)
print(len(correct_toks))
print(inputs)
print(len(inputs))

[['‘', 'You', 'know', ',', 'the', 'panda', 'who', 'keeps', 'an', 'eye', 'on', 'my', 'drinking', 'habits', '.', '’'], ['But', 'I', 'will', 'also', 'be', 'keeping', 'an', 'eye', 'on', 'you', '.', '’'], ['‘', 'I', 'will', 'keep', 'an', 'eye', 'on', 'him', ',', '’', 'reassured', 'Jack', '.'], ['Moreover', ',', 'international', 'operations', 'director', 'Richard', 'Ferry', 'claims', 'that', 'the', 'Bracknell', ',', 'Berkshire-based', 'company', "'s", 'services', 'are', 'unique', '—', 'although', 'most', 'rivals', 'can', 'offer', 'centralised', 'expertise', ',', 'he', 'says', ',', 'no-one', 'else', 'can', 'monitor', 'computer', 'systems', 'remotely', ',', 'or', 'keep', 'an', 'eye', 'on', 'such', 'critical', 'issues', 'as', 'temperature', 'and', 'air', 'conditioning', 'levels', 'at', 'any', 'given', 'customer', 'site', '.'], ['Once', 'Duncan', 'had', 'managed', 'to', 'fire', 'off', 'a', 'report', 'and', 'stop', 'his', 'watch-replacement', 'from', 'stumbling', 'on', 'the', 'terrorists', ',', '

Generating inputs and correct tokens: 2761it [00:00, 99850.60it/s]

['on', 'on', 'on', 'on', 'on', 'on', 'on', 'on', 'on', 'on', 'on', 'together', 'together', 'together', 'together', 'open', 'open', 'open', 'open', 'open', 'open', 'open', 'open', 'open', 'open', 'open', 'open', 'open', 'open', 'open', 'open', 'open', 'open', 'peeled', 'peeled', 'peeled', 'peeled', 'peeled', 'peeled', 'on', 'on', 'on', 'on', 'on', 'on', 'on', 'on', 'on', 'on', 'door', 'door', 'door', 'door', 'door', 'door', 'door', 'door', 'water', 'water', 'water', 'water', 'water', 'water', 'water', 'water', 'water', 'water', 'water', 'bucket', 'bucket', 'bucket', 'bucket', 'time', 'time', 'time', 'time', 'day', 'day', 'eyelid', 'eyelid', 'eyelid', 'eyelid', 'health', 'health', 'health', 'health', 'health', 'health', 'health', 'health', 'health', 'health', 'horse', 'horse', 'horses', 'loss', 'loss', 'loss', 'Loss', 'Loss', 'work', 'work', 'WORK', 'work', 'work', 'work', 'work', 'trick', 'trick', 'trick', 'feet', 'feet', 'feet', 'feet', 'feet', 'feet', 'feet', 'feet', 'feet', 'feet', '

In [ ]:
# Test prompts
# Optional
#pythia_14m.generate("Which part of the sentence ‘You know, the panda who keeps an eye on my drinking habits.’ is part of an idiom?") # Macht falsch weiter
#pythia_14m.generate("Input: 'I kick the bucket .'\nOutput: '0 B-IDIOM I-IDIOM I-IDIOM I-IDIOM 0'\nInput: ‘You know, the panda who keeps an eye on my drinking habits.’\nOutput: ") # gibt nicht den erwünschten Output
#pythia_14m.generate("‘You know, the panda who keeps an eye") # macht falsch weiter
out_tensor = pythia_14m.generate("‘You know, the panda who keeps an eye", max_new_tokens = 1, verbose = False, return_type = "tokens")
pred = pythia_14m.to_str_tokens(out_tensor)[-1].strip()

 on
on


In [ ]:
# Single example
out_tensor = pythia_14m.generate(pythia_14m.to_tokens(inputs[0]), max_new_tokens = 1, verbose = False, return_type = "tokens")
print(out_tensor[:,1:])
pred = pythia_14m.to_str_tokens(out_tensor)[-1].strip()
print(pred)

tensor([[ 8924,  1394,   871,    13,   253,   268,  7447,   665, 11359,   271,
          5130,   327]], device='cuda:0')
on


In [ ]:
def predict_idiom(model):
  print("Model:", model.cfg.model_name)
  num_correct = 0
  incorrect_answers = []
  correct_answers = []

  for (pos, input) in tqdm(enumerate(inputs), desc = "Generating predictions"):
    #print(input)
    out_tensor = model.generate(model.to_tokens(input), max_new_tokens = 1, verbose = False, return_type = "tokens")[:, 1:]
    pred = model.to_str_tokens(out_tensor)[-1].strip()

    if pred == correct_toks[pos]:
      num_correct += 1

      if len(correct_answers) < 5:
        correct_answers.append(model.to_string(out_tensor)[0])
    else:
      if len(incorrect_answers) < 5:
        incorrect_answers.append((input + " " + correct_toks[pos], model.to_string(out_tensor)[0]))

  print(f"\nThe model generated {num_correct} correct answers.")
  print("Accuracy: ", "{:.2%}".format(num_correct/len(inputs)))
  print(f"Five correct outputs: {correct_answers}")
  print(f"Five incorrect outputs: {incorrect_answers}")

In [ ]:
predict_idiom(pythia_14m)

Model: pythia-14m


Generating predictions: 2761it [00:39, 69.51it/s]


The model generated 54 correct answers.
Accuracy:  1.96%
Five correct outputs: ["Moreover, international operations director Richard Ferry claims that the Bracknell, Berkshire-based company's services are unique — although most rivals can offer centralised expertise, he says, no-one else can monitor computer systems remotely, or keep an eye on", 'Once Duncan had managed to fire off a report and stop his watch-replacement from stumbling on the terrorists, he had settled down to keep an eye on', 'So we must keep an eye on', 'Keep it in the quarantine tank, cease medication and keep an eye on', '‘As you can see from the state of this place, I have someone call in regularly to keep an eye on']
Five incorrect outputs: [('‘You know, the panda who keeps an eye on', '‘You know, the panda who keeps an eye peeled'), ('But I will also be keeping an eye on', 'But I will also be keeping an eye for'), ('‘I will keep an eye on', '‘I will keep an eye out'), ('Mary organized the plates and kept an eye

In [ ]:
predict_idiom(pythia_70m)

Model: pythia-70m


Generating predictions: 2761it [00:41, 65.82it/s]


The model generated 359 correct answers.
Accuracy:  13.00%
Five correct outputs: ['‘You know, the panda who keeps an eye on', 'But I will also be keeping an eye on', '‘I will keep an eye on', "Moreover, international operations director Richard Ferry claims that the Bracknell, Berkshire-based company's services are unique — although most rivals can offer centralised expertise, he says, no-one else can monitor computer systems remotely, or keep an eye on", 'Once Duncan had managed to fire off a report and stop his watch-replacement from stumbling on the terrorists, he had settled down to keep an eye on']
Five incorrect outputs: [('‘As you can see from the state of this place, I have someone call in regularly to keep an eye on', '‘As you can see from the state of this place, I have someone call in regularly to keep an eye out'), ('Mary organized the plates and kept an eye on', 'Mary organized the plates and kept an eye out'), ('Keep an eye on', 'Keep an eye.'), ('Much of the owner-occup

In [ ]:
predict_idiom(pythia_1_4b)

Model: pythia-1.4b


Generating predictions: 2761it [02:58, 15.49it/s]


The model generated 1191 correct answers.
Accuracy:  43.14%
Five correct outputs: ['‘You know, the panda who keeps an eye on', '‘I will keep an eye on', "Moreover, international operations director Richard Ferry claims that the Bracknell, Berkshire-based company's services are unique — although most rivals can offer centralised expertise, he says, no-one else can monitor computer systems remotely, or keep an eye on", 'Once Duncan had managed to fire off a report and stop his watch-replacement from stumbling on the terrorists, he had settled down to keep an eye on', 'So we must keep an eye on']
Five incorrect outputs: [('But I will also be keeping an eye on', 'But I will also be keeping an eye out'), ("I will take some Bovril up for'er too, and some sausage rolls, we all'ave to keep body an' soul together", "I will take some Bovril up for'er too, and some sausage rolls, we all'ave to keep body an' soul go"), ('Their education, he said, was irrelevant to their duties, their poverty such

In [ ]:
# geht nicht, weil Colab
pythia_12b: HookedTransformer = HookedTransformer.from_pretrained("EleutherAI/pythia-12b")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]